In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import os
import json
import sys
import numpy as np
sys.path.append("..")
import copy
from tqdm.auto import tqdm

In [ ]:
from src import models, data, operators, utils, functional, metrics, lens
from src.utils import logging_utils, experiment_utils
import logging
import torch
import baukit

experiment_utils.set_seed(123456)

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format = logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout
)

In [ ]:
from src.functional import save_order_1_approx, normalize_on_sphere

### Interpolation Experiment

In [ ]:
mt = models.load_model(name = "gptj", fp16 = True, device = "cuda")

In [ ]:
relation_name = "name birthplace"
relation = data.load_dataset().filter(relation_names=[relation_name])[0]

In [ ]:
def load_cached_values(trial_path, idx):
    approx = np.load(f"{trial_path}/approx_{idx+1}.npz", allow_pickle=True)
    approx_dict = {}
    for key,value in approx.items():
        if key in ["h", "z", "weight", "bias"]:
            approx_dict[key] = torch.from_numpy(value).cuda()
        else:
            approx_dict[key] = value
    weight = torch.Tensor(approx['weight']).cuda()
    h = torch.Tensor(approx['h']).cuda()
    z = torch.Tensor(approx['z']).cuda()

    return weight, h, z, approx

In [ ]:
from src.operators import JacobianIclMeanEstimator

In [ ]:
def execute_line_integral_experiment(trial_path, interpolation_steps, lre_W = None, h_layer = None):
    jacobians, hs, zs = [], [], []
    for idx in tqdm(range(interpolation_steps)):
        j, h, z, approx  = load_cached_values(trial_path, idx)
        jacobians.append(j)
        hs.append(h)
        zs.append(z)

        if h_layer is None:
            h_layer = approx["h_layer"].item()
    
    u = zs[-1] - zs[0]
    v = hs[-1] - hs[0]
    u /= u.norm()
    v /= v.norm()

    dh = (hs[-1] - hs[0])/interpolation_steps

    scalar_values = []
    Dz = zs[-1] - zs[0]
    Dz_approx = 0
    for dF in jacobians:
        s = u[None] @ dF @ v[None].T
        scalar_values.append(s.item())
        Dz_approx += s * dh

    if lre_W is None:
        estimator = JacobianIclMeanEstimator(mt = mt, h_layer=h_layer)
        train, test = relation.split(train_size=5)
        operator = estimator(train)
        lre_W = operator.weight.float()

    Dz_lre = (u[None] @ lre_W @ v[None].T) * (hs[-1] - hs[0]).norm()

    return Dz.norm().item(), Dz_approx.norm().item(), Dz_lre.norm().item(), scalar_values

In [ ]:
# s1, s2 = "Brazil", "Spain"


experiment_utils.set_seed(123456)

path = f"../results/interpolation/{relation_name}/"
trials = os.listdir(path)

actual_delta_zs = []
approx_delta_zs = []
lre_delta_zs = []
scalar_values = []

for trial in trials: #[f"{s1}-{s2}"]:
    print(trial)
    trial_path = os.path.join(path, trial)
    interpolation_steps = len(os.listdir(trial_path))
    Dz, Dz_approx, Dz_lre, values_i = execute_line_integral_experiment(trial_path, interpolation_steps)

    actual_delta_zs.append(Dz)
    approx_delta_zs.append(Dz_approx)
    lre_delta_zs.append(Dz_lre)
    scalar_values.append(values_i)

    plt.plot(np.linspace(0, 1, interpolation_steps), values_i, label=trial)
    print(f"True Dz: {Dz:.4f} | Approx Dz: {Dz_approx:.4f} | LRE Dz: {Dz_lre:.4f}")


plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel("Interpolation ratio, $\\alpha$")
plt.ylabel(r"$u^T \times F^'(s) \times v$")

In [ ]:
beta = torch.Tensor(approx_delta_zs)/torch.Tensor(lre_delta_zs)
f"beta: {beta.mean():.3f}+/-{beta.std():.3f}"

In [ ]:
fig_dir = "figures/figs"
#####################################################################################
plt.rcdefaults()
color = "purple"
plt.rcParams["figure.dpi"] = 200
plt.rcParams["font.family"] = "Times New Roman"

SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", labelsize=MEDIUM_SIZE+1)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=50)  # fontsize of the figure title
#####################################################################################

In [ ]:
scalar_values_mean = torch.Tensor(scalar_values).mean(dim=0)
scalar_values_std = torch.Tensor(scalar_values).std(dim=0)

lre_delta_mean = torch.Tensor(lre_delta_zs).mean()
lre_delta_std = torch.Tensor(lre_delta_zs).std()

plt.plot(np.linspace(0, 1, interpolation_steps), scalar_values_mean, color="purple", label = r"$u^T \; F^'(s) \; \Delta s$")
plt.fill_between(np.linspace(0, 1, interpolation_steps), scalar_values_mean - scalar_values_std, scalar_values_mean + scalar_values_std, color="purple", alpha=0.1)
plt.axhline(y=lre_delta_mean/100, color='r', linestyle='--', label=r"$u^T \; W \; \Delta s$")

plt.legend()
plt.xlabel("Interpolation ratio, $\\alpha$")
plt.xlim(0, 1)
plt.ylim(bottom=0)

plt.savefig(f"{fig_dir}/line_integration.pdf", bbox_inches="tight")

In [ ]:
from src.utils.sweep_utils import read_sweep_results, relation_from_dict

sweep_path = f"../results/sweep-24-trials/gptj"
relation_result_raw = read_sweep_results(
    sweep_path, relation_names=[relation_name], economy=True
)[relation_name]
relation_result = relation_from_dict(relation_result_raw)

In [ ]:
relation_result.best_by_efficacy()

### Loading Cached $\mathcal{W}$ and $b$

In [ ]:
# relation = data.load_dataset().filter(relation_names=["country capital city"])[0].set(prompt_templates=[" {}:"])
# train, test = relation.split(5)

# #################################################
# h_layer = 8
# beta = 2.25
# interpolation_steps = 100
# #################################################

# mt = models.load_model(name = "gptj", fp16 = True, device = "cuda")

In [ ]:
# path = "../results/interpolation/country capital city/"
# W_norms = []
# B_norms = []
# Jh_norms = []
# J_delta_h = []

# values = []
# labels = []


# for trial in os.listdir(path):
#     # if(trial == "Colombia-Venezuela"):
#     #     continue
#     if(trial != f"{s1}-{s2}"):
#         continue

#     s1, s2 = trial.split("-")
#     hs_and_zs = functional.compute_hs_and_zs(
#         mt = mt,
#         prompt_template = train.prompt_templates[0],
#         subjects = list(set([sample.subject for sample in relation.samples] + [s1, s2])),
#         h_layer= h_layer,
#         z_layer=-1,
#         examples= train.samples
#     )
#     h1, h2 = [hs_and_zs.h_by_subj[s] for s in [s1, s2]]
#     z1, z2 = [hs_and_zs.z_by_subj[s] for s in [s1, s2]]

#     do = (z2 - z1).float()
#     ds = (h2 - h1).float()

#     trial_path = os.path.join(path, trial)
#     approxes = []
#     interpolation_steps = len(os.listdir(trial_path))
#     for idx in tqdm(range(interpolation_steps)):
#         approx = np.load(f"{trial_path}/approx_{idx+1}.npz", allow_pickle=True)
#         approx_dict = {}
#         for key,value in approx.items():
#             if key in ["h", "z", "weight", "bias"]:
#                 approx_dict[key] = torch.from_numpy(value).cuda()
#             else:
#                 approx_dict[key] = value
#         approxes.append(approx_dict)

#     for a in approxes:
#         weight = torch.Tensor(a['weight']).float().cuda()

#         # scalar_value = torch.cosine_similarity(weight @ ds, do, dim = -1)
#         # scalar_value = torch.dot(weight @ ds, do)
#         # G = torch.tensor([torch.dot(do, weight[i]) for i in range(models.determine_hidden_size(mt))]).cuda()
#         # scalar_value = torch.dot(G, ds)
#         scalar_value = do[None] @ (weight @ ds[None].T)
#         values.append(scalar_value.item())



#     # w_norms = [torch.Tensor(a['weight']).norm().item() for a in approxes]
#     # b_norms = [torch.Tensor(a['bias']).norm().item() for a in approxes]
#     # jh_norms = [torch.Tensor(a["metadata"].item()["Jh"]).norm().item() for a in approxes]

#     # h1 = normalize_on_sphere(approxes[0]['h'], scale=60.0)
#     # h2 = normalize_on_sphere(approxes[-1]['h'], scale=60.0)
#     # delta_h = normalize_on_sphere(h2 - h1, scale=60.0)
#     # # delta_h = hs_and_zs.h_by_subj["Russia"]
#     # j_delta_h = [(torch.Tensor(a['weight']) @ delta_h).norm().item() for a in approxes]

#     # W_norms.append(w_norms)
#     # B_norms.append(b_norms)
#     # Jh_norms.append(jh_norms)
#     # J_delta_h.append(j_delta_h)
#     # labels.append(trial)

In [ ]:
# plt.rcdefaults()

# plt.plot(np.linspace(0, 1, interpolation_steps), values)

In [ ]:
# plt.rcdefaults()

# for trial, label in zip(W_norms, labels):
#     plt.plot(np.linspace(0, 1, interpolation_steps), trial, label=label)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.ylabel("|| $W$ ||")

In [ ]:
# # filter_trials = ["South Korea-Brazil", "Brazil-Nigeria", "Egypt-South Korea", "Chile-Egypt"]
# filter_trials = ["Brazil-Turkey", "Turkey-France", "Brazil-France", "Spain-Egypt"]
# trial_results = []
# for trial, label in zip(W_norms, labels):
#     if label not in filter_trials:
#         continue
#     trial_results.append(trial)
#     # plt.plot(np.linspace(0, 1, interpolation_steps), trial, label=label)
# # plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# trial_results = np.array(trial_results)
# trial_results_mean = trial_results.mean(axis=0)
# trial_results_std = trial_results.std(axis=0)


# fig_dir = "figures/figs"
# #####################################################################################
# plt.rcdefaults()
# color = "purple"
# plt.rcParams["figure.dpi"] = 200
# plt.rcParams["font.family"] = "Times New Roman"

# SMALL_SIZE = 12
# MEDIUM_SIZE = 16
# BIGGER_SIZE = 18

# plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
# plt.rc("axes", labelsize=MEDIUM_SIZE+1)  # fontsize of the x and y labels
# plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
# plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
# plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
# plt.rc("figure", titlesize=50)  # fontsize of the figure title
# #####################################################################################

# x = np.linspace(0, 1, interpolation_steps)
# plt.plot(x, trial_results_mean, color=color)
# plt.fill_between(x, trial_results_mean - trial_results_std, trial_results_mean + trial_results_std, alpha=0.1, color = color)
# plt.xlabel("Interpolation ratio, $\\alpha$")
# plt.ylabel("|| $W$ ||")

# plt.savefig(f"{fig_dir}/j_underestimation.pdf", bbox_inches="tight")

In [ ]:
# # J(h2 - h1)
# plt.rcdefaults()
# for trial, label in zip(J_delta_h, labels):
#     plt.plot(np.linspace(0, 1, interpolation_steps), trial, label=label)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.ylabel("|| $W @ \Delta h$ ||")

In [ ]:
# for trial in Jh_norms:
#     plt.plot(np.linspace(0, 1, interpolation_steps), trial)